<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Algorithmic Trading 

**Chapter 08 &mdash; CFD Trading with Oanda**

## The Oanda API

`pip install git+https://github.com/yhilpisch/tpqoa.git`

In [4]:
import tpqoa

In [5]:
api = tpqoa.tpqoa('../pyalgo.cfg')  # adjust the path/filename

## Retrieving Historical Data

### Looking Up Instruments Available for Trading

In [6]:
api.get_instruments()[:15]

ResponseNoField: 400 response for GET https://api-fxpractice.oanda.com:443/v3/accounts/mawan@icloud.com/instruments does not have field 'instruments' (contains 'errorMessage')

### Backtesting a Momentum Strategy on Minute Bars

In [ ]:
help(api.get_history)

In [ ]:
instrument = 'EUR_USD'
start = '2020-05-27'
end = '2020-05-29'
granularity = 'M1'
price = 'M'

In [ ]:
data = api.get_history(instrument, start, end,
                      granularity, price)

In [ ]:
data.info()

In [ ]:
data[['c', 'volume']].head()

In [ ]:
import numpy as np

In [ ]:
data['returns'] = np.log(data['c'] / data['c'].shift(1))

In [ ]:
cols = []

In [ ]:
for momentum in [15, 30, 60, 120]:
    col = 'position_{}'.format(momentum)
    data[col] = np.sign(data['returns'].rolling(momentum).mean())
    cols.append(col)

In [ ]:
from pylab import plt
plt.style.use('seaborn-v0_8')
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'

In [ ]:
strats = ['returns']

In [ ]:
for col in cols:
    strat = 'strategy_{}'.format(col.split('_')[1])
    data[strat] = data[col].shift(1) * data['returns']
    strats.append(strat)

In [ ]:
data[strats].dropna().cumsum(
    ).apply(np.exp).plot(figsize=(10, 6));

### Factoring In Leverage and Margin

In [ ]:
data[strats].dropna().cumsum().apply(
            lambda x: x * 20).apply(np.exp).plot(figsize=(10, 6));

## Working with Streaming Data

In [ ]:
instrument = 'EUR_USD'

In [ ]:
api.stream_data(instrument, stop=10)

## Placing Orders

In [ ]:
help(api.create_order)

In [ ]:
api.create_order(instrument, 1000)

In [ ]:
api.create_order(instrument, -1500)

In [ ]:
api.create_order(instrument, 500)

## Implementing Trading Strategies in Real-Time

In [ ]:
import MomentumTrader as MT

In [ ]:
mt = MT.MomentumTrader('../pyalgo.cfg',
                       instrument=instrument,
                       bar_length='15s',
                       momentum=6,
                       units=10000)

In [ ]:
mt.stream_data(mt.instrument, stop=150)

In [ ]:
oo = mt.create_order(instrument, units=-mt.position * mt.units,
                     ret=True, suppress=True)
oo

### Retrieving Account Information

In [ ]:
api.get_account_summary()

In [ ]:
api.get_transactions(tid=int(oo['id']) - 2)

In [ ]:
api.print_transactions(tid=int(oo['id']) - 18)

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>